In [91]:
import sys
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../file_loader")
sys.path.append("../preprocessing-block")
sys.path.append("../features_calculation")


import ecg_file_loader
from ecg_file_loader import ECGFileLoader

import rpeaks_finder
from rpeaks_finder import *

import ectopic_beats_filtering
from ectopic_beats_filtering import *

import time_features
from time_features import *

reload(ecg_file_loader)
reload(rpeaks_finder)
reload(ectopic_beats_filtering)
reload(time_features)

<module 'time_features' from '/Users/pietrosaveri/Desktop/Pietro/•StartUps/Seizury/• src_blocks/check_codes/../features_calculation/time_features.py'>

In [2]:
loader = ECGFileLoader(use_s3=False)

In [3]:
processed_data = process_all_patients_rpeaks(loader, ['sub-001'])

INFO:rpeaks_finder:Processing R-peaks detection for 1 patients: ['sub-001']
INFO:rpeaks_finder:Loading data for patient sub-001
INFO:ecg_file_loader:Processing sub-001 run 01
INFO:ecg_file_loader:Processing sub-001 run 02
INFO:ecg_file_loader:Processing sub-001 run 03
INFO:ecg_file_loader:Processing sub-001 run 04
INFO:ecg_file_loader:Processing sub-001 run 05
INFO:ecg_file_loader:Processing sub-001 run 06
INFO:ecg_file_loader:Processing sub-001 run 07
INFO:ecg_file_loader:Processing sub-001 run 08
INFO:ecg_file_loader:Processing sub-001 run 09
INFO:rpeaks_finder:Processing sub-001 run 01
INFO:rpeaks_finder:ECG signal shape: (16721664,), sampling rate: 256.0 Hz
INFO:rpeaks_finder:Successfully processed sub-001 run 01: 72450 R-peaks detected
INFO:rpeaks_finder:Processing sub-001 run 02
INFO:rpeaks_finder:ECG signal shape: (5072896,), sampling rate: 256.0 Hz
INFO:rpeaks_finder:Successfully processed sub-001 run 02: 22104 R-peaks detected
INFO:rpeaks_finder:Processing sub-001 run 03
INFO:

In [4]:
processed_rr = process_dvc_for_session(processed_data['sub-001'][0], min_rr_s=0.3, max_rr_s=1.3)


--------------------------------------------------
Processing session 01 for patient sub-001
--------------------------------------------------
Initial state: 72449 RRs, 72450 timestamps.
--- Starting Ectopic Beat Filtering ---
Found 57 RR intervals > 1.3s. Deleting them to create gaps.
Found short RR interval (0.289s) at index 49661. Analyzing merge options...
  > No merge met all conditions. Deleting the single short RR interval as an artifact.
Found short RR interval (0.293s) at index 31502. Analyzing merge options...
  > Right merge successful. New RR: 0.648s.
Found short RR interval (0.273s) at index 31486. Analyzing merge options...
  > No merge met all conditions. Deleting the single short RR interval as an artifact.
Found short RR interval (0.270s) at index 16261. Analyzing merge options...
  > Right merge successful. New RR: 1.078s.
Found short RR interval (0.289s) at index 16227. Analyzing merge options...
  > No merge met all conditions. Deleting the single short RR interva

In [5]:
rr_intervals = processed_rr['dvc_rr_intervals_ms']
timestamps = processed_rr['dvc_rpeak_times_s']

In [54]:
len(timestamps[1:]), len(rr_intervals)

(72506, 72506)

### Test with just one to see if it works

In [ ]:
import pyhrv.time_domain as td

results = td.time_domain(rr_intervals, sampling_rate = 512, plot = False)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyhrv/time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '


In [ ]:
#non ci sono RR_var e SampEn
results

ReturnTuple(nni_counter=72506, nni_mean=900.8188223216009, nni_min=300.78125, nni_max=1281.25, hr_mean=67.21942162954706, hr_min=46.829268292682926, hr_max=199.4805194805195, hr_std=6.843016245975114, nni_diff_mean=22.327949132064308, nni_diff_min=0, nni_diff_max=921, sdnn=83.1034264484127, sdnn_index=33.5247358746849, sdann=73.67596725420566, rmssd=34.13911017714668, sdsd=25.825389250296475, nn50=5066, pnn50=6.987104337631887, nn20=30507, pnn20=42.07571891593683, nni_histogram=None, tinn_n=0, tinn_m=0, tinn=0, tri_index=26.01578758521708)

### Test on sliding windows

In [64]:
features = compute_hrv_features(rr_intervals, timestamps, sampling_rate=512, window_minutes=5)

for i, window in enumerate(features):
    start = window['start_time_s']
    end = window['end_time_s']
    feats = window['features']
    print(f"\nWindow {i+1}: {start:.2f}s → {end:.2f}s")
    print(feats)


Window 1: 0.00s → 300.15s
ReturnTuple(nni_counter=362, nni_mean=829.0422133977901, nni_min=808.59375, nni_max=855.46875, hr_mean=72.38247871717643, hr_min=70.13698630136986, hr_max=74.20289855072464, hr_std=0.8422858729036607, nni_diff_mean=9.954986149584487, nni_diff_min=0, nni_diff_max=31, sdnn=9.680400798378688, sdnn_index=7.6084122505453475, sdann=0.6427130542295024, rmssd=12.187294683617543, sdsd=7.040291441757016, nn50=0, pnn50=0.0, nn20=28, pnn20=7.756232686980609, nni_histogram=None, tinn_n=796.875, tinn_m=851.5625, tinn=54.6875, tri_index=3.480769230769231)

Window 2: 300.15s → 599.76s
ReturnTuple(nni_counter=357, nni_mean=839.2528886554621, nni_min=796.875, nni_max=875.0, hr_mean=71.50710334831103, hr_min=68.57142857142857, hr_max=75.29411764705883, hr_std=1.0352513660335603, nni_diff_mean=10.522735252808989, nni_diff_min=0, nni_diff_max=46, sdnn=12.152960360558794, sdnn_index=nan, sdann=nan, rmssd=12.689825007848233, sdsd=7.102492667844786, nn50=0, pnn50=0.0, nn20=17, pnn20

In [70]:
features[0]['features'] #prima window

ReturnTuple(nni_counter=362, nni_mean=829.0422133977901, nni_min=808.59375, nni_max=855.46875, hr_mean=72.38247871717643, hr_min=70.13698630136986, hr_max=74.20289855072464, hr_std=0.8422858729036607, nni_diff_mean=9.954986149584487, nni_diff_min=0, nni_diff_max=31, sdnn=9.680400798378688, sdnn_index=7.6084122505453475, sdann=0.6427130542295024, rmssd=12.187294683617543, sdsd=7.040291441757016, nn50=0, pnn50=0.0, nn20=28, pnn20=7.756232686980609, nni_histogram=None, tinn_n=796.875, tinn_m=851.5625, tinn=54.6875, tri_index=3.480769230769231)

### Test on 1 minute "pakages"

In [ ]:
# CI METTE UN SACCO

features_sliding = compute_hrv_features_sliding(
    rr_intervals=rr_intervals,
    timestamps=timestamps,
    sampling_rate=512,
    window_minutes=5,
    step_minutes=1
)

for f in features_sliding[:5]:
    print(f"\nMinute {f['minute']:.0f}: {f['start_time_s']:.2f}s → {f['end_time_s']:.2f}s")
    print(f['features'])


In [ ]:
# SO CHE PROBABILEMTE ROMBO UN RR-INTERAL MA ERA SOLO PER PROVARE SE IL WINDOWING ERA GIUSTO

rr_intervals_short = rr_intervals[:5000]
timestamps_short = timestamps[:5001]

In [ ]:
features_sliding = compute_hrv_features_sliding(
    rr_intervals=rr_intervals_short,
    timestamps=timestamps_short,
    sampling_rate=512,
    window_minutes=5,
    step_minutes=1
)

for f in features_sliding[:5]:
    print(f"\nMinute {f['minute']:.0f}: {f['start_time_s']:.2f}s → {f['end_time_s']:.2f}s")
    print(f['features'])


Starting sliding HRV computation...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyhrv/utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyhrv/time_domain.py:303: UserWarning: Signal duration too short for SDANN computation.
  warnings.warn("Signal duration too short for SDANN computation.")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyhrv/time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '


Finished processing.

Minute 5: 0.86s → 300.15s
ReturnTuple(nni_counter=361, nni_mean=829.0555747922438, nni_min=808.59375, nni_max=855.46875, hr_mean=72.3813326512107, hr_min=70.13698630136986, hr_max=74.20289855072464, hr_std=0.8431722111909327, nni_diff_mean=9.971788194444445, nni_diff_min=0, nni_diff_max=31, sdnn=9.69049324717567, sdnn_index=nan, sdann=nan, rmssd=12.202473107321149, sdsd=7.042838368532329, nn50=0, pnn50=0.0, nn20=28, pnn20=7.777777777777778, nni_histogram=None, tinn_n=796.875, tinn_m=851.5625, tinn=54.6875, tri_index=3.4711538461538463)

Minute 6: 60.30s → 360.33s
ReturnTuple(nni_counter=361, nni_mean=831.1114958448753, nni_min=808.59375, nni_max=867.1875, hr_mean=72.20320586390595, hr_min=69.1891891891892, hr_max=74.20289855072464, hr_std=0.879664365504865, nni_diff_mean=9.971788194444445, nni_diff_min=0, nni_diff_max=31, sdnn=10.163313327402383, sdnn_index=9.231875010236834, sdann=4.795374763906201, rmssd=12.233694837947398, sdsd=7.096945604831086, nn50=0, pnn50=

In [ ]:
features_sliding[0] #prima window

{'minute': 5.002473958333334,
 'start_time_s': 0.859375,
 'end_time_s': 300.1484375,
 'features': ReturnTuple(nni_counter=361, nni_mean=829.0555747922438, nni_min=808.59375, nni_max=855.46875, hr_mean=72.3813326512107, hr_min=70.13698630136986, hr_max=74.20289855072464, hr_std=0.8431722111909327, nni_diff_mean=9.971788194444445, nni_diff_min=0, nni_diff_max=31, sdnn=9.69049324717567, sdnn_index=nan, sdann=nan, rmssd=12.202473107321149, sdsd=7.042838368532329, nn50=0, pnn50=0.0, nn20=28, pnn20=7.777777777777778, nni_histogram=None, tinn_n=796.875, tinn_m=851.5625, tinn=54.6875, tri_index=3.4711538461538463)}